In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer
from env import user, password, host
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import os
import csv
import wrangle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, TweedieRegressor, LassoLars
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# the target will be taxamount

In [3]:
zillow_train,zillow_validate,zillow_test=wrangle.wrangled_file()

In [4]:
zillow_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25350 entries, 39641 to 7566
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   bathroom                    25350 non-null  float64
 1   bedroom                     25350 non-null  float64
 2   calculatedbathnbr           25350 non-null  float64
 3   sqtft                       25350 non-null  float64
 4   finishedsquarefeet12        25350 non-null  float64
 5   fips                        25350 non-null  object 
 6   fullbathcnt                 25350 non-null  float64
 7   latitude                    25350 non-null  float64
 8   longitude                   25350 non-null  float64
 9   lots                        25350 non-null  float64
 10  propertycountylandusecode   25350 non-null  object 
 11  propertyzoningdesc          25350 non-null  object 
 12  rawcensustractandblock      25350 non-null  float64
 13  zipcode                     

In [5]:
zillow_train.T

,39641,59493,9448,69877,49854,56795,55632,33533,17385,7099,...,36748,61398,64005,72249,8119,66181,9172,61419,36522,7566
bathroom,3.0,2.0,2.0,1.0,2.0,1.0,3.0,2.0,2.0,1.0,...,2.0,2.0,1.0,2.0,1.0,3.0,3.0,1.0,2.0,2.0
bedroom,3.0,3.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0,2.0,...,3.0,3.0,2.0,3.0,2.0,2.0,3.0,5.0,3.0,2.0
calculatedbathnbr,3.0,2.0,2.0,1.0,2.0,1.0,3.0,2.0,2.0,1.0,...,2.0,2.0,1.0,2.0,1.0,3.0,3.0,1.0,2.0,2.0
sqtft,1882.0,2257.0,1470.0,970.0,1268.0,1118.0,1127.0,2346.0,1794.0,788.0,...,1720.0,1289.0,1902.0,1565.0,1296.0,1354.0,1860.0,3018.0,1820.0,1345.0
finishedsquarefeet12,1882.0,2257.0,1470.0,970.0,1268.0,1118.0,1127.0,2346.0,1794.0,788.0,...,1720.0,1289.0,1902.0,1565.0,1296.0,1354.0,1860.0,3018.0,1820.0,1345.0
fips,6037.0,6037.0,6037.0,6037.0,6037.0,6037.0,6037.0,6037.0,6037.0,6037.0,...,6037.0,6037.0,6037.0,6037.0,6037.0,6037.0,6037.0,6037.0,6037.0,6037.0
fullbathcnt,3.0,2.0,2.0,1.0,2.0,1.0,3.0,2.0,2.0,1.0,...,2.0,2.0,1.0,2.0,1.0,3.0,3.0,1.0,2.0,2.0
latitude,33890566.0,33818762.0,34230800.0,34009255.0,34067106.0,34045899.0,34416000.0,34165174.0,34699995.0,34089049.0,...,34100885.0,34106232.0,34042879.0,34709719.0,34079733.0,34184000.0,34544862.0,34000306.0,33880946.0,34019500.0
longitude,-118154278.0,-118322465.0,-118532000.0,-118069242.0,-118135667.0,-118003252.0,-118448000.0,-118641275.0,-118161110.0,-118330531.0,...,-117885893.0,-117865961.0,-118353121.0,-118166943.0,-118306745.0,-118598000.0,-118069054.0,-118293834.0,-118396519.0,-117813000.0
lots,15090.0,6826.0,13230.0,6563.0,6017.0,6355.0,76716.0,11251.0,7172.0,2600.0,...,6912.0,7807.0,5527.0,7212.0,5723.0,224297.0,8119.0,8173.0,3436.0,179588.0


In [6]:
x_train, y_train = zillow_train.drop(columns='logerror'),zillow_train.logerror
x_validate, y_validate = zillow_validate.drop(columns='logerror'),zillow_validate.logerror
x_test, y_test = zillow_test.drop(columns='logerror'),zillow_test.logerror

In [7]:
x_train=x_train.drop(columns=['propertycountylandusecode','propertyzoningdesc','heatingorsystemdesc','propertylandusedesc','county'])
x_validate=x_validate.drop(columns=['propertycountylandusecode','propertyzoningdesc','heatingorsystemdesc','propertylandusedesc','county'])
x_test=x_test.drop(columns=['propertycountylandusecode','propertyzoningdesc','heatingorsystemdesc','propertylandusedesc','county'])             

In [28]:
scaler=MinMaxScaler()
x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train), index=x_train.index, columns=x_train.columns)
x_validate_scaled = pd.DataFrame(scaler.fit_transform(x_validate), index=x_validate.index, columns=x_validate.columns)
x_test_scaled = pd.DataFrame(scaler.fit_transform(x_test), index=x_test.index, columns=x_test.columns)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [29]:
# We need y_train and y_validate to be dataframes to append the new columns with predicted values. 
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)

# predict mean
y_train['baseline'] = y_train['logerror'].mean()
y_validate['baseline'] = y_validate['logerror'].mean()

# predict median
y_train['logerror_med'] = y_train['logerror'].median()
y_validate['logerror_med'] = y_validate['logerror'].median()

# RMSE of mean
rmse_train = mean_squared_error(y_train.logerror, y_train.baseline)**(1/2)
rmse_validate = mean_squared_error(y_validate.logerror, y_validate.baseline)**(1/2)

print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

# RMSE of median
rmse_train = mean_squared_error(y_train.logerror, y_train.logerror_med)**(1/2)
rmse_validate = mean_squared_error(y_validate.logerror, y_validate.logerror_med)**(1/2)

print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))



RMSE using Mean
Train/In-Sample:  0.15 
Validate/Out-of-Sample:  0.16
RMSE using Median
Train/In-Sample:  0.15 
Validate/Out-of-Sample:  0.16


In [30]:
models = pd.DataFrame(
[
    {
        'model': 'baseline',
        'rmse': mean_squared_error(y_train['logerror'], y_train.baseline,squared=False),
        'r^2': explained_variance_score(y_train['logerror'], y_train.baseline)
    
    }
])
models

,model,rmse,r^2
0,baseline,0.150015,0.0


In [31]:
def modeling(model, 
                  x_train, 
                  y_train, 
                  x_validate, 
                  y_validate, 
                  scores=models):
    model.fit(x_train, y_train.logerror)
    in_sample_pred = model.predict(x_train)
    out_sample_pred = model.predict(x_validate)
    model_name = input('model name?')
    y_train[model_name] = in_sample_pred
    y_validate[model_name] = out_sample_pred
    print(y_validate.shape)
    print(out_sample_pred.shape)
    rmse_val = mean_squared_error(
    y_validate.logerror, out_sample_pred, squared=False)
    r_squared_val = explained_variance_score(
        y_validate.logerror, out_sample_pred)
    return models.append({
        'model': model_name,
        'rmse': rmse_val,
        'r^2': r_squared_val
    
    }, ignore_index=True)

In [32]:
models = modeling(LinearRegression(normalize=True), 
                  x_train_scaled, 
                  y_train, 
                  x_validate_scaled, 
                  y_validate, 
                  scores=models)

model name?
(10787, 9)
(10787,)


In [33]:
models = modeling(LassoLars(alpha=1.0), 
                  x_train_scaled, 
                  y_train, 
                  x_validate_scaled, 
                  y_validate, 
                  scores=models)

model name?
(10787, 9)
(10787,)


In [34]:
polyfeats = PolynomialFeatures(degree=2)
x_train_quad = polyfeats.fit_transform(x_train_scaled)
x_val_quad = polyfeats.transform(x_validate_scaled)
models = modeling(LinearRegression(), 
                  x_train_quad, 
                  y_train, 
                  x_val_quad, 
                  y_validate, 
                  scores=models)

model name?
(10787, 9)
(10787,)


In [35]:
models = modeling(TweedieRegressor(power=0, alpha=0), 
                  x_train_scaled, 
                  y_train, 
                  x_validate_scaled, 
                  y_validate, 
                  scores=models)

model name?
(10787, 9)
(10787,)


In [36]:
models

,model,rmse,r^2
0,baseline,1.500153e-01,0.000000e+00
1,,9.733785e+09,-2.249241e+21
2,,1.626836e-01,0.000000e+00
3,,4.926769e+08,-6.552863e+18
4,,1.621130e-01,6.151741e-03


In [37]:
x_train_scaled

,bathroom,bedroom,calculatedbathnbr,sqtft,finishedsquarefeet12,fips,fullbathcnt,latitude,longitude,lots,...,yearbuilt,structuretaxvaluedollarcnt,taxvalue,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,taxrate,month,age
39641,0.2,0.272727,0.2,0.075621,0.075621,0.0,0.2,0.372515,0.596852,0.003999,...,0.832117,0.019121,0.007292,0.0,0.002280,0.008693,1.071292e-05,0.026876,0.500,0.167883
59493,0.1,0.272727,0.1,0.092913,0.092913,0.0,0.1,0.323974,0.461345,0.001696,...,0.627737,0.022792,0.009948,0.0,0.004397,0.010816,1.301030e-05,0.024718,0.750,0.372263
9448,0.1,0.272727,0.1,0.056624,0.056624,0.0,0.1,0.602521,0.292525,0.003481,...,0.927007,0.032312,0.014621,0.0,0.006657,0.015574,3.334176e-07,0.024335,0.125,0.072993
69877,0.0,0.272727,0.0,0.033568,0.033568,0.0,0.0,0.452751,0.665364,0.001623,...,0.583942,0.011136,0.013746,0.0,0.015513,0.015687,9.447188e-06,0.026039,0.875,0.416058
49854,0.1,0.272727,0.1,0.047309,0.047309,0.0,0.1,0.491860,0.611847,0.001471,...,0.554745,0.027611,0.021984,0.0,0.019644,0.022775,9.007025e-06,0.023747,0.625,0.445255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66181,0.2,0.181818,0.2,0.051275,0.051275,0.0,0.2,0.570883,0.239350,0.062300,...,0.722628,0.021772,0.015445,0.0,0.012901,0.016520,8.045278e-07,0.024449,0.875,0.277372
9172,0.2,0.272727,0.2,0.074607,0.074607,0.0,0.2,0.814834,0.665516,0.002057,...,0.839416,0.019108,0.007288,0.0,0.002280,0.010926,1.915506e-05,0.033583,0.125,0.160584
61419,0.0,0.454545,0.0,0.128003,0.128003,0.0,0.0,0.446702,0.484413,0.002072,...,0.204380,0.021673,0.026494,0.0,0.029062,0.028372,3.101554e-06,0.024576,0.750,0.795620
36522,0.1,0.272727,0.1,0.072762,0.072762,0.0,0.1,0.366012,0.401681,0.000752,...,0.620438,0.043995,0.071660,0.0,0.084249,0.068737,1.229812e-05,0.022059,0.500,0.379562


In [38]:
kbest = SelectKBest(f_regression, k=3)
kbest.fit(x_train_scaled, y_train.logerror)
mask = x_train_scaled.columns[kbest.get_support()].to_list()

In [39]:
models = modeling(LinearRegression(), 
                  x_train_scaled[mask], 
                  y_train, 
                  x_validate_scaled[mask], 
                  y_validate, 
                  scores=models)

model name?
(10787, 9)
(10787,)


In [40]:
models

,model,rmse,r^2
0,baseline,1.500153e-01,0.000000e+00
1,,9.733785e+09,-2.249241e+21
2,,1.626836e-01,0.000000e+00
3,,4.926769e+08,-6.552863e+18
4,,1.621130e-01,6.151741e-03
5,,1.625070e-01,1.594428e-03


In [41]:
models['r^2'].max()

0.006151741446707826

In [22]:
polyfeats = PolynomialFeatures(degree=2)
x_test_quad = polyfeats.fit_transform(x_test_scaled)

In [23]:
def modeling(model, 
                  x_test, 
                  y_test, 
                  scores=models):
    model.fit(x_test, y_test)
    in_sample_pred = model.predict(x_test)
    model_name = input('model_name?')
    y_test[model_name] = in_sample_pred
    rmse_val = mean_squared_error(
    y_test, in_sample_pred, squared=False)**(1/2)
    r_squared_val = explained_variance_score(
        y_test, in_sample_pred)
    return models.append({
        'model': model_name,
        'rmse': rmse_val,
        'r^2': r_squared_val
    
    }, ignore_index=True)

In [24]:
x_test_scaled=x_test_scaled.dropna()
x_test_scaled

,bathroom,bedroom,calculatedbathnbr,sqtft,finishedsquarefeet12,fips,fullbathcnt,latitude,longitude,lots,...,yearbuilt,structuretaxvaluedollarcnt,taxvalue,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,taxrate,month,age
15991,-0.241832,1.044492,-0.241832,0.432787,0.432787,0.0,-0.241832,0.266778,0.706322,-0.283107,...,-1.400919,0.257775,0.306007,0.0,0.299421,0.262739,0.034353,-0.483090,-0.867915,1.400919
70854,-0.241832,-0.966296,-0.241832,-0.771086,-0.771086,0.0,-0.241832,0.154563,0.006205,0.617035,...,0.853788,-0.093607,-0.277262,0.0,-0.330032,-0.340884,-0.595351,-0.648335,1.224602,-0.853788
60923,-0.241832,0.039098,-0.241832,-0.183749,-0.183749,0.0,-0.241832,-0.293418,-0.738785,-0.309771,...,-1.574358,-0.541319,-0.460510,0.0,-0.386221,-0.481523,-0.730966,-0.078331,0.806099,1.574358
35827,-1.264035,-0.966296,-1.264035,-0.638571,-0.638571,0.0,-1.264035,-0.397883,0.496830,-0.315608,...,-0.577084,-0.464380,-0.272796,0.0,-0.168472,-0.257818,0.306365,0.006949,-0.030908,0.577084
74842,-0.241832,-0.966296,-0.241832,-1.072055,-1.072055,0.0,-0.241832,-0.074551,1.317784,0.180837,...,0.767069,-0.244901,-0.309823,0.0,-0.309907,-0.326620,-0.192055,-0.230695,1.643105,-0.767069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63973,-0.241832,0.039098,-0.241832,-0.266852,-0.266852,0.0,-0.241832,0.432595,-0.801524,-0.302704,...,-0.490364,-0.556190,-0.521607,0.0,-0.461363,-0.551196,-1.284016,-0.083778,1.224602,0.490364
62489,-0.241832,-0.966296,-0.241832,-0.594773,-0.594773,0.0,-0.241832,-0.373576,1.612934,-0.296952,...,0.290111,-0.512636,-0.522191,0.0,-0.480419,-0.527588,-0.179351,0.578966,0.806099,-0.290111
17683,-0.241832,0.039098,-0.241832,-0.475733,-0.475733,0.0,-0.241832,0.610239,-0.732772,-0.293027,...,-0.490364,-0.382930,-0.357954,0.0,-0.316090,-0.384137,-1.315854,-0.287903,-0.867915,0.490364
56270,-0.241832,-0.966296,-0.241832,0.067806,0.067806,0.0,-0.241832,0.388031,0.576954,-0.236891,...,-1.400919,-0.309608,-0.256615,0.0,-0.211876,-0.272454,0.025313,-0.259435,0.806099,1.400919


In [25]:
y_test

15991    0.024403
70854   -0.040040
60923   -0.065035
35827    0.030651
74842    0.031033
           ...   
63973    0.030080
62489    0.267016
17683    0.049941
56270    0.072429
37917    0.003813
Name: logerror, Length: 8923, dtype: float64

In [26]:
polyfeats = PolynomialFeatures(degree=2)
modeltest=LinearRegression()
modeltest.fit(x_test, y_test)
x_test = polyfeats.fit_transform(x_test_scaled)

In [27]:
rmse_val = mean_squared_error(
    y_test, modeltest.predict(x_test), squared=False)
r_squared_val = explained_variance_score(
        y_test, modeltest.predict(x_test))

ValueError: X has 325 features, but LinearRegression is expecting 24 features as input.

In [ ]:
rmse_val,r_squared_val